In [1]:
# 영화리뷰 긍/부정 분류때 학습시켜 사용했던 모델을 이용해 blog 검색결과를 긍/부정으로 분류하고
# 각 확률을 점수화해보기
# naver_search_blog에서 저장한 파일을 이용해서 웹툰별 블로그 글 (최대 100)개에 대해 긍/부정 점수를 매긴다.
# 1. positive와 negative로 분류해서 점수매긴 파일
# 2. positive는 높은 점수, negative는 낮은 점수로 합산해 단일점수를 매긴 파일
import pandas as pd
import numpy as np
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [3]:
print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력

훈련용 리뷰 개수 : 150000


In [4]:
train_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거

In [5]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거

In [19]:
text = 'do!!! you expect... people~ to~ read~ the FAQ, etc. and actually accept hard~! atheism?@@'
re.sub(r'[^a-zA-Z ]', '', text) #알파벳과 공백을 제외하고 모두 제거

'do you expect people to read the FAQ etc and actually accept hard atheism'

In [6]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",regex=True)
# 한글과 공백을 제외하고 모두 제거
train_data[:5]

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [7]:
train_data['document'] = train_data['document'].str.replace('^ +', "", regex = True) # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
#기존에 한글이 없던 리뷰는 빈값이 되었을 것이므로 다시 null값 찾아야함

In [8]:
train_data = train_data.dropna(how = 'any')

In [9]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",regex = True) # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "", regex = True) # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거

In [10]:
#불용어 제거
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [11]:
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

['오다', '이렇다', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만들다', '게', '나다', '뻔']

In [12]:
X_train = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [13]:
X_test = []
for sentence in test_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

In [14]:
# 정수 인코딩
# vocaburary 만들기
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [15]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 43752
등장 빈도가 2번 이하인 희귀 단어의 수: 24337
단어 집합에서 희귀 단어의 비율: 55.62488571950996
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.8715872104872904


In [16]:
# 희귀단어의 비율은 전체의 절반 이상인것에 비해 총 희귀단어 등장 빈도는 1.8%이므로 자연어 처리에 중요해보이지 않음
# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 19416


In [17]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [18]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [19]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [20]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)

C:\Users\Jeongyeop\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [22]:
max_len = 30
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [23]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [24]:
loaded_model = load_model('movieReview_best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1527/1527 [==============================] - 20s 13ms/step - loss: 0.3315 - acc: 0.8555

 테스트 정확도: 0.8555


In [25]:
def sentiment_predict(new_sentence):
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
        #print(\"{:.2f}% 확률로 긍정 리뷰입니다.\\n\".format(score * 100))
        return score*100
    else:
        #print(\"{:.2f}% 확률로 부정 리뷰입니다.\\n\".format((1 - score) * 100))
        return (1-score)*-100

In [27]:
# naver_search_blog에서 저장한 파일
import json
with open('data/NaverWebtoonData.json', encoding='utf-8') as jsonFile:
    jsonData = json.load(jsonFile)

In [31]:
newList = []
for i in range(len(jsonData)):
    newDict = {}
    cnt_p = 0
    cnt_n = 0
    total_p = 0
    total_n = 0
    #print(jsonData[i]['title'])
    length = len(jsonData[i]['blog_description'])
    #print(f"총 {length}개의 데이터가 있습니다.")
    for str in jsonData[i]['blog_description']:
        PN = sentiment_predict(str)
        if PN > 0:
            cnt_p += 1
            total_p += PN
        else :
            cnt_n += 1
            total_n -= PN
    newDict['title'] = jsonData[i]['title']
    newDict['nSearch'] = cnt_p + cnt_n
    if cnt_p != 0 :
        #print(f"총 {cnt_p}개의 긍정리뷰가 있고 평균확률은 {total_p//cnt_p} 입니다.")
        newDict['positive review'] = int(total_p)
    else :
        newDict['positive review'] = 0
    if cnt_n != 0 :
        #print(f"총 {cnt_n}개의 부정리뷰가 있고 평균확률은 {total_n//cnt_n} 입니다.")
        newDict['negative review'] = int(total_n)
    else :
        newDict['negative review'] = 0
    
    newList.append(newDict)

In [32]:
#json 파일로 저장
file_path = 'data/dataPN.json'
with open(file_path, 'w', encoding = "utf-8") as outfile:
    outfile.write(json.dumps(newList, ensure_ascii=False))

In [33]:
newList[:2]

[{'title': '가비지타임',
  'nSearch': 100,
  'positive review': 5513,
  'negative review': 2546},
 {'title': '겟백',
  'nSearch': 100,
  'positive review': 4825,
  'negative review': 3093}]

In [34]:
def sentiment_predict_score(new_sentence):
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
        return score*100
    else:
        return score*100

In [35]:
newList = []
for i in range(len(jsonData)):
    newDict = {}
    cnt = 0
    totalScore = 0
    length = len(jsonData[i]['blog_description'])
    for str in jsonData[i]['blog_description']:
        score = sentiment_predict_score(str)
        cnt += 1
        totalScore += score

    newDict['title'] = jsonData[i]['title']
    if cnt != 0 :
        newDict['nSearch'] = cnt
        newDict['review_score'] = int(totalScore)
    else :
        newDict['nSearch'] = 0
        newDict['review_score'] = 0

    
    newList.append(newDict)

In [36]:
#json 파일로 저장
file_path = 'data/dataReviewScore.json'
with open(file_path, 'w', encoding = "utf-8") as outfile:
    outfile.write(json.dumps(newList, ensure_ascii=False))